In [ ]:
import xarray as xr

from src import tools

In [ ]:
fix_boundaries = False
mask_bad_area = False
filter_by_depth = True

In [ ]:
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid_version3.nc')

## Fix mask_rho variable - boundaries check

In [ ]:
if fix_boundaries:
    grid_handler = tools.GridHandler(grid_ds)
    west_idx = grid_handler.find_matches('west')
    north_idx = grid_handler.find_matches('north')
    east_idx = grid_handler.find_matches('east')
    south_idx = grid_handler.find_matches('south')
    
    grid_handler.print_values(west_idx, north_idx, east_idx, south_idx)
    
    # Get xi and eta coordinates to use loc assignment (isel or iloc won't work)
    xi_rho_west_coords = grid_ds.coords['xi_rho'][0]
    eta_rho_west_coords = grid_ds.coords['eta_rho'][west_idx.squeeze()] * xr.ones_like(xi_rho_west_coords)
    print(f"Xi west coordinates: {xi_rho_west_coords}\n",
          f"Eta west coordinates: {eta_rho_west_coords}\n")
    
    xi_rho_north_coords = grid_ds.coords['xi_rho'][north_idx.squeeze()]
    eta_rho_north_coords = grid_ds.coords['eta_rho'][-1] * xr.ones_like(xi_rho_north_coords)
    print(f"Xi north coordinates: {xi_rho_north_coords}\n",
          f"Eta north coordinates: {eta_rho_north_coords}\n")
    
    xi_rho_east_coords = grid_ds.coords['xi_rho'][-1]
    eta_rho_east_coords = grid_ds.coords['eta_rho'][east_idx.squeeze()] * xr.ones_like(xi_rho_east_coords)
    print(f"Xi east coordinates: {xi_rho_east_coords}\n",
          f"Eta east coordinates: {eta_rho_east_coords}\n")
    
    xi_rho_south_coords = grid_ds.coords['xi_rho'][south_idx.squeeze()]
    eta_rho_south_coords = grid_ds.coords['eta_rho'][0] * xr.ones_like(xi_rho_south_coords)
    print(f"Xi south coordinates: {xi_rho_south_coords}\n",
          f"Eta south coordinates: {eta_rho_south_coords}\n")

    # mask the corresponding points in mask_rho
    coords = (
        (xi_rho_west_coords, eta_rho_west_coords),
        (xi_rho_north_coords, eta_rho_north_coords),
        (xi_rho_east_coords, eta_rho_east_coords),
        (xi_rho_south_coords, eta_rho_south_coords),
    )

    for coord in coords:
        grid_ds.mask_rho.loc[dict(xi_rho=coord[0], eta_rho=coord[1])] = 0
    
    grid_handler.print_values(west_idx, north_idx, east_idx, south_idx)

## Mask out the particular points, which cause instabilities

In [ ]:
if mask_bad_area:
    grid_ds.mask_rho.isel(xi_rho=slice(1434, None), eta_rho=slice(685, 698)).plot(figsize=(14, 7))
    grid_ds.mask_rho.values[685:698, 1434:] = 0
    grid_ds.mask_rho.isel(xi_rho=slice(1434, None), eta_rho=slice(685, 698)).plot(figsize=(14, 7))


## Apply depth filter

In [ ]:
if filter_by_depth:
    grid_ds.h.plot(figsize=(14, 7))  # without figsize (both) there will be one picture
    grid_ds = tools.GridHandler.filter_min_depth(grid_ds, 50)
    grid_ds.h.plot(figsize=(14, 7))
    tools.GridHandler.to_netcdf(grid_ds, "v3+")